<a href="https://colab.research.google.com/github/Alijgh003/StyleMind-GNN/blob/main/clipGarmentEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
from os import path

In [12]:
datasets_path = path.join("/","content","drive","MyDrive","StyleMind-GNN")
images_path = path.join(datasets_path, "dataset", "polyvore_outfits", "images")
edge_indices_path = path.join(datasets_path, "dataset", "polyvore_outfits", "nondisjoint")

In [9]:
import pandas as pd

In [11]:
items_df = pd.read_csv(path.join(datasets_path, "new_items.csv"))
items_df

,Unnamed: 0.1,Unnamed: 0,index,ID,url_name,category_id,semantic_category,fdescription
0,0,0,0,211990161,neck print chiffon plus size,3,tops,neck print chiffon plus size tops
1,1,1,1,183179503,christian pellizzari floral jacquard trousers,2,bottoms,christian pellizzari floral jacquard trousers ...
2,2,2,2,152771755,natures jewelry stainless steel not,11,jewellery,natures jewelry stainless steel not jewellery
3,3,3,3,190445143,balmain women high waist flared,2,bottoms,balmain women high waist flared Decorative gol...
4,4,4,4,211444470,acler orson high waist belted,2,bottoms,acler orson high waist belted This Acler Orson...
...,...,...,...,...,...,...,...,...
251003,251003,251003,251003,108411005,arnica embellished hand painted skirt,2,bottoms,arnica embellished hand painted skirt Shop Ste...
251004,251004,251004,251004,212452593,thom browne little boy knitted,4,outerwear,thom browne little boy knitted outerwear
251005,251005,251005,251005,208651882,colorful vortex print shawl scarf,8,scarves,colorful vortex print shawl scarf scarves
251006,251006,251006,251006,209210426,floral embroidered mesh see thru,3,tops,floral embroidered mesh see thru tops


In [14]:
import torch

In [30]:
def build_edge_indices_with_groups(edge_index_tensor_by_groups):
  if(edge_index_tensor_by_groups.shape[0]>0):
    result = torch.cat([edge_index_tensor_by_groups[i] for i in range(edge_index_tensor_by_groups.shape[0])],dim=-1)
    result = result[:,torch.where(result[0]>-1)[0]]
    return result
  else:
    return torch.randn((2, 1))

In [31]:
def build_edge_indices_and_indices_with_groups(edge_index_tensor_by_groups):
  if(edge_index_tensor_by_groups.shape[0]>0):
    edge_indices = torch.cat([edge_index_tensor_by_groups[i] for i in range(edge_index_tensor_by_groups.shape[0])],dim=-1)
    indices = torch.cat([torch.full((edge_index_tensor_by_groups[i].shape[-1],), i) for i in range(edge_index_tensor_by_groups.shape[0])], dim=-1)
    indices = indices[edge_indices[0]>=0].view(-1,1)
    edge_indices = edge_indices[edge_indices>=0].view(2,-1)
    return edge_indices, indices
  else:
    return torch.randn((2, 1)), torch.randn((2,1))

In [34]:
train_edge_index = torch.load(path.join(edge_indices_path, "train_edge_index.pth"))
negative_train_edge_index = torch.load(path.join(edge_indices_path,"negative_train_edge_index.pth"))

train_edge_index  = build_edge_indices_with_groups(train_edge_index)
negative_train_edge_index  = build_edge_indices_with_groups(negative_train_edge_index)
train_edge_index, negative_train_edge_index

(tensor([[204439, 154376, 154376,  ...,  74651,  74651,  74651],
         [249298, 249298, 204439,  ..., 145194, 153601, 220923]]),
 tensor([[118742, 118742, 100723,  ...,  64782,  64782,  64782],
         [178245, 100723, 178245,  ...,  37821,   4739,  57581]]))

In [ ]:
test_edge_index = torch.load(path.join(edge_indices_path, "test_edge_index.pth"))
valid_edge_index = torch.load(path.join(edge_indices_path, "valid_edge_index.pth"))


valid_negative_samples = torch.load(path.join(edge_indices_path, "negative_valid_edge_index.pth"))
test_negative_samples = torch.load(path.join(edge_indices_path, "negative_test_edge_index.pth"))

negative_train_edge_index.shape , valid_negative_samples.shape, test_negative_samples.shape

(torch.Size([53306, 2, 171]),
 torch.Size([5000, 2, 91]),
 torch.Size([10000, 2, 136]))

In [16]:
train_edge_index.shape, test_edge_index.shape, valid_edge_index.shape

(torch.Size([53306, 2, 171]),
 torch.Size([10000, 2, 136]),
 torch.Size([5000, 2, 91]))

In [22]:
build_edge_indices_and_indices_with_groups(train_edge_index)[:100]

(tensor([[204439, 154376, 154376,  ...,  74651,  74651,  74651],
         [249298, 249298, 204439,  ..., 145194, 153601, 220923]]),
 tensor([[    0],
         [    0],
         [    0],
         ...,
         [53305],
         [53305],
         [53305]]))

In [24]:
!pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=7d14a6f1e35566e3d6694169cae97998a6fd5606ff9971e8d6318deba94bb033
  Stored in directory: /root/.cache/pip/wheels/ab/a5/e8/c9fa20742edbccf2702dae8ee62053e6c460e961d45967b49c
Successfully built clip


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import clip
from torch.utils.data import Dataset, DataLoader

In [27]:
class FusionHead(nn.Module):
    def __init__(self, embed_dim, num_of_layers=1):
        super(FusionHead, self).__init__()
        self.fcs = torch.nn.ModuleList([nn.Linear(2 * embed_dim, embed_dim)] + [nn.Linear(embed_dim, embed_dim) for _ in range(num_of_layers-1)])
        self.bns = torch.nn.ModuleList([nn.BatchNorm1d(embed_dim)] + [nn.BatchNorm1d(embed_dim) for _ in range(num_of_layers-1)])
        self.activation = nn.ReLU()

    def reset_parameters(self):
        for fc in self.fcs:
            fc.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for fc, bn in zip(self.fcs, self.bns):
            x = self.activation(bn(fc(x)))
        return x

In [29]:
from PIL import Image

In [ ]:
class OutfitDataset(Dataset):
    def __init__(self, data, preprocess):
        self.data = data  # a list of (img_a_path, txt_a, img_b_path, txt_b, label)
        self.preprocess = preprocess

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_a_path, txt_a, img_b_path, txt_b, label = self.data[idx]

        img_a = self.preprocess(Image.open(img_a_path))
        img_b = self.preprocess(Image.open(img_b_path))

        return img_a, txt_a, img_b, txt_b, label


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import clip
from torch.utils.data import Dataset, DataLoader

# ====================
# 1. Define Fusion Head
# ====================
class FusionHead(nn.Module):
    def __init__(self, embed_dim):
        super(FusionHead, self).__init__()
        self.fc = nn.Linear(2 * embed_dim, embed_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x)
        return x

# ====================
# 2. Define a Dataset
# ====================
class OutfitDataset(Dataset):
    def __init__(self, data, preprocess):
        self.data = data  # a list of (img_a_path, txt_a, img_b_path, txt_b, label)
        self.preprocess = preprocess

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_a_path, txt_a, img_b_path, txt_b, label = self.data[idx]

        img_a = self.preprocess(Image.open(img_a_path))
        img_b = self.preprocess(Image.open(img_b_path))

        return img_a, txt_a, img_b, txt_b, label

# ====================
# 3. Load CLIP Model
# ====================
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Optional: freeze some layers if needed
# for param in model.parameters():
#     param.requires_grad = False

# Create fusion head
fusion_head = FusionHead(embed_dim=model.visual.output_dim).to(device)

# ====================
# 4. Prepare Data
# ====================
from PIL import Image

# Example data format
train_data = [
    ("path/to/img1.jpg", "red hoodie", "path/to/img2.jpg", "blue jeans", 1),
    ("path/to/img3.jpg", "white shirt", "path/to/img4.jpg", "black shorts", 0),
    # add more samples...
]

train_dataset = OutfitDataset(train_data, preprocess)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ====================
# 5. Training Setup
# ====================
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(fusion_head.parameters()),
    lr=1e-5
)
loss_fn = nn.CosineEmbeddingLoss(margin=0.5)

# ====================
# 6. Training Loop
# ====================
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    fusion_head.train()

    epoch_loss = 0.0
    for img_a, txt_a, img_b, txt_b, labels in train_loader:
        img_a, img_b = img_a.to(device), img_b.to(device)
        labels = labels.to(device).float()

        # Encode images and texts
        img_emb_a = model.encode_image(img_a)
        txt_emb_a = model.encode_text(clip.tokenize(txt_a).to(device))

        img_emb_b = model.encode_image(img_b)
        txt_emb_b = model.encode_text(clip.tokenize(txt_b).to(device))

        # Concatenate and fuse
        combined_a = torch.cat([img_emb_a, txt_emb_a], dim=-1)
        combined_b = torch.cat([img_emb_b, txt_emb_b], dim=-1)

        emb_a = fusion_head(combined_a)
        emb_b = fusion_head(combined_b)

        # Normalize embeddings (optional but good)
        emb_a = F.normalize(emb_a, p=2, dim=1)
        emb_b = F.normalize(emb_b, p=2, dim=1)

        # Compute loss
        targets = 2 * labels - 1  # 1 for positive, -1 for negative
        loss = loss_fn(emb_a, emb_b, targets)

        # Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss/len(train_loader):.4f}")

print("Training complete!")
